In [1]:
import keras
from keras.layers import Input, Dense, Lambda, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
import tensorflow as tf

from models import vgg
from utils.angles import deg2bit, bit2deg
from utils.losses import maad_from_deg 
from utils.losses import von_mises_log_likelihood_tf
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yte_bit = deg2bit(yte_deg)

In [4]:
#x_tf = tf.placeholder(tf.float32, shape=[None, 3, image_width, image_height])
mu_pred_tf = tf.placeholder(tf.float32, shape=[None, 2])
kappa_tf = tf.placeholder(tf.float32, shape=[None, 1])
y_true_tf = tf.placeholder(tf.float32, shape=[None, 2])
y_pred_tf = tf.concat([mu_pred_tf, kappa_tf], axis=1)

In [53]:
from functools import partial

In [54]:
vm_likelihood_keras = partial(von_mises_neg_log_likelihood_keras, fixed_kappa=0.5)

In [76]:
mu_kappa

<tf.Tensor 'concatenate_11/concat:0' shape=(?, 3) dtype=float32>

In [81]:
X = Input(shape=[50,50,3])
vgg_x = vgg.vgg_model(final_layer=False,
                      image_height=50,
                      image_width=50)(X)

mu = Lambda(lambda x: K.l2_normalize(x, axis=1))(Dense(2)(vgg_x))

kappa_pred = tf.ones([tf.shape(mu)[0],1]) #Input(shape=[1]) # # 
kappa_pred = Dense(1)(vgg_x)

#mu_kappa = concatenate([mu, kappa])

def von_mises_neg_log_likelihood_keras(y_true, y_pred):
    mu_pred = y_pred[:,0:2]
#     if fixed_kappa!=0:
#         kappa_pred = tf.ones(shape=tf.shape(y_pred[:, 2:]))*fixed_kappa
#     else:
#         kappa_pred = tf.abs(y_pred[:, 2:])
    return -von_mises_log_likelihood_tf(y_true, mu_pred, kappa_pred, input_type='biternion')

biternion_vgg_prob = Model(X, mu)

optimizer = keras.optimizers.Adam(lr=0.0001)

biternion_vgg_prob.compile(loss=von_mises_neg_log_likelihood_keras,
                           metrics=['cosine'],
                           optimizer='adam')

In [ ]:
biternion_vgg_prob.fit(x=xtr , y=ytr_bit, batch_size=10, validation_split=0.1, epochs=20)

Train on 7002 samples, validate on 778 samples
Epoch 1/20
7002/7002 [==============================] - 78s - loss: 1.7140 - cosine_proximity: -0.2026 - val_loss: 1.6964 - val_cosine_proximity: -0.2119
Epoch 2/20
7002/7002 [==============================] - 74s - loss: 1.5493 - cosine_proximity: -0.2712 - val_loss: 1.7303 - val_cosine_proximity: -0.2085
Epoch 3/20
7002/7002 [==============================] - 74s - loss: 1.4406 - cosine_proximity: -0.3129 - val_loss: 1.6846 - val_cosine_proximity: -0.2211
Epoch 4/20
 960/7002 [===>..........................] - ETA: 62s - loss: 1.4781 - cosine_proximity: -0.3026

In [45]:
mu_kappa = biternion_vgg_prob.predict(xte)
yte_preds = bit2deg(mu_kappa[:,0:2])
kappa_te_preds = mu_kappa[:,2]

In [46]:
np.abs(kappa_te_preds)

array([  3.48741627e+00,   7.61694098e+00,   1.61925049e+01,
         1.53622169e+01,   6.18109322e+00,   7.15980911e+00,
         6.69455099e+00,   3.78344727e+00,   8.95522881e+00,
         6.00795364e+00,   3.06068420e-01,   4.54397106e+00,
         1.39271059e+01,   1.50732880e+01,   1.66020870e-01,
         2.76904774e+00,   1.13971734e+00,   5.41366291e+00,
         2.96658039e+00,   1.53837109e+00,   4.97174740e+00,
         8.52370644e+00,   4.06093311e+00,   4.06585932e+00,
         8.97528172e+00,   2.58592749e+00,   4.90848255e+00,
         3.41501641e+00,   1.89839625e+00,   1.07824821e+01,
         2.47121811e-01,   4.90017891e-01,   2.67152762e+00,
         7.41273499e+00,   7.40206766e+00,   1.73727179e+00,
         2.29601173e+01,   6.49566650e+00,   4.51702738e+00,
         4.19950867e+00,   3.80398571e-01,   1.61215997e+00,
         1.08638077e+01,   9.96006489e-01,   5.47716331e+00,
         5.13088274e+00,   5.93051386e+00,   4.58122730e+00,
         1.48096561e+00,

In [51]:
mu_kappa[:,0]**2+mu_kappa[:,1]**2

array([ 0.99999994,  0.99999988,  0.99999988,  0.9999997 ,  0.99999976,
        0.99999982,  0.99999994,  0.99999982,  1.        ,  0.99999982,
        1.        ,  0.99999988,  0.99999982,  0.99999988,  1.        ,
        1.        ,  1.        ,  1.00000012,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.00000012,  1.        ,
        0.99999988,  1.        ,  0.99999988,  1.        ,  1.        ,
        0.99999988,  0.99999988,  0.99999988,  0.99999982,  0.99999994,
        0.99999988,  1.00000012,  0.99999994,  0.99999994,  0.99999982,
        1.00000012,  0.99999988,  1.00000012,  1.        ,  1.        ,
        0.99999994,  0.99999988,  1.        ,  0.99999994,  1.        ,
        1.        ,  1.        ,  0.99999982,  0.99999982,  1.00000024,
        0.99999994,  1.        ,  1.        ,  1.00000012,  1.        ,
        0.99999988,  1.        ,  0.99999994,  1.        ,  1.00000012,
        0.99999988,  0.99999994,  0.99999994,  1.        ,  1.00

In [47]:
loss = maad_from_deg(yte_preds, yte_deg)

mean_loss = np.mean(loss)
std_loss = np.std(loss)

print("MAAD error (test) : %f ± %f" % (mean_loss, std_loss))

MAAD error (test) : 34.471532 ± 35.834381
